In [118]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable

In [119]:
dataset = pd.read_csv('/content/drive/My Drive/toy/Churn_Modelling.csv')

In [120]:
# dataset.drop(['RowNumber'],axis = 1, inplace = True)

In [121]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [122]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [123]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [124]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [125]:
# Normalization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [126]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [127]:
y_train

array([0, 0, 0, ..., 0, 0, 1])

In [128]:
y = torch.from_numpy(y_train)
y.shape

torch.Size([8000])

In [129]:
# y = y.unsqueeze(1)
# y

In [130]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
)
simple_net

Sequential(
  (0): Linear(in_features=784, out_features=30, bias=True)
  (1): ReLU()
  (2): Linear(in_features=30, out_features=1, bias=True)
)

In [131]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(12, 512)         # obviamente tem que bater com as features, e a input
        self.fc2 = nn.Linear(512, 512)        # escolhemos aleatoriamente, sao as hidden features
        self.fc3 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
model = Net()
model

Net(
  (fc1): Linear(in_features=12, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [137]:
[param.shape for param in model.parameters()]

[torch.Size([512, 12]),
 torch.Size([512]),
 torch.Size([512, 512]),
 torch.Size([512]),
 torch.Size([2, 512]),
 torch.Size([2])]

In [140]:
for name, param in model.named_parameters():
  print(name, param.shape)

fc1.weight torch.Size([512, 12])
fc1.bias torch.Size([512])
fc2.weight torch.Size([512, 512])
fc2.bias torch.Size([512])
fc3.weight torch.Size([2, 512])
fc3.bias torch.Size([2])


In [132]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

batch_size = 64
n_epochs = 500
batch_no = len(X_train) // batch_size

train_loss = 0
train_loss_min = np.Inf
for epoch in range(n_epochs):
    for i in range(batch_no):
        start = i*batch_size
        end = start+batch_size
        x_var = Variable(torch.FloatTensor(X_train[start:end]))
        y_var = Variable(torch.LongTensor(y_train[start:end])) 
        
        optimizer.zero_grad()
        output = model(x_var)
        loss = criterion(output,y_var)
        loss.backward()
        optimizer.step()
        
        values, labels = torch.max(output, 1)
        num_right = np.sum(labels.data.numpy() == y_train[start:end])
        train_loss += loss.item()*batch_size
    
    train_loss = train_loss / len(X_train)
    if train_loss <= train_loss_min:
        print("Validation loss decreased ({:6f} ===> {:6f}). Saving the model...".format(train_loss_min,train_loss))
        torch.save(model.state_dict(), "model.pt")
        train_loss_min = train_loss
    
    if epoch % 200 == 0:
        print('')
        print("Epoch: {} \tTrain Loss: {} \tTrain Accuracy: {}".format(epoch+1, train_loss,num_right / len(y_train[start:end]) ))
print('Training Ended! ')

Validation loss decreased (   inf ===> 0.504480). Saving the model...

Epoch: 1 	Train Loss: 0.504479662656784 	Train Accuracy: 0.796875
Validation loss decreased (0.504480 ===> 0.454679). Saving the model...
Validation loss decreased (0.454679 ===> 0.439198). Saving the model...
Validation loss decreased (0.439198 ===> 0.429281). Saving the model...
Validation loss decreased (0.429281 ===> 0.425218). Saving the model...
Validation loss decreased (0.425218 ===> 0.421305). Saving the model...
Validation loss decreased (0.421305 ===> 0.419486). Saving the model...
Validation loss decreased (0.419486 ===> 0.414196). Saving the model...
Validation loss decreased (0.414196 ===> 0.409597). Saving the model...
Validation loss decreased (0.409597 ===> 0.404711). Saving the model...
Validation loss decreased (0.404711 ===> 0.402154). Saving the model...
Validation loss decreased (0.402154 ===> 0.397180). Saving the model...
Validation loss decreased (0.397180 ===> 0.395800). Saving the model...

In [133]:
# class Model(nn.Module):
#     def __init__(self, num_features, num_targets, hidden_size):
#         super(Model, self).__init__()
#         self.batch_norm1 = nn.BatchNorm1d(num_features)
#         self.dropout1 = nn.Dropout(0.2)
#         self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
#         self.batch_norm2 = nn.BatchNorm1d(hidden_size)
#         self.dropout2 = nn.Dropout(0.5)
#         self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
#         self.batch_norm3 = nn.BatchNorm1d(hidden_size)
#         self.dropout3 = nn.Dropout(0.5)
#         self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
#     def forward(self, x):
#         x = self.batch_norm1(x)
#         x = self.dropout1(x)
#         x = F.relu(self.dense1(x))
        
#         x = self.batch_norm2(x)
#         x = self.dropout2(x)
#         x = F.relu(self.dense2(x))
        
#         x = self.batch_norm3(x)
#         x = self.dropout3(x)
#         x = self.dense3(x)
        
#         return x